환경설정


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

참고문서: https://python.langchain.com/docs/modules/data_connection/document_transformers/


## 텍스트 분할기

문서를 로드한 후에는 애플리케이션에 더 적합하도록 문서를 변형하고 싶을 때가 많습니다. 가장 간단한 예로, 긴 문서를 모델의 컨텍스트 창에 맞도록 작은 덩어리로 분할하고 싶을 수 있습니다.

LangChain에는 문서를 쉽게 분할, 결합, 필터링 및 기타 조작할 수 있는 여러 가지 문서 변환기가 내장되어 있습니다.

긴 텍스트를 다루고 싶을 때는 해당 텍스트를 여러 조각으로 분할해야 합니다. 간단하게 들리지만 여기에는 많은 잠재적 복잡성이 있습니다. 이상적으로는 의미적으로 관련된 텍스트 조각을 함께 보관하는 것이 좋습니다. "의미적으로 연관된"이란 텍스트 유형에 따라 달라질 수 있습니다. 이 노트북에서는 이를 위한 몇 가지 방법을 보여드립니다.

큰 틀에서 텍스트 분할기는 다음과 같이 작동합니다.

- 텍스트를 의미적으로 의미 있는 작은 덩어리(주로 문장)로 나눕니다.
- 특정 크기(특정 함수로 측정한 크기)에 도달할 때까지 이 작은 청크들을 더 큰 청크로 결합하기 시작합니다.
- 그 크기에 도달하면 그 청크를 자체 텍스트 조각으로 만든 다음, 청크 사이의 맥락을 유지하기 위해 약간의 겹침이 있는 새로운 텍스트 청크를 만들기 시작합니다.

즉, 텍스트 분할기를 사용자 지정할 수 있는 두 개의 다른 축이 있다는 뜻입니다.

- 텍스트 분할 방식
- 청크 크기를 측정하는 방법


## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`


## PDF 문서 로드를 위한 다양한 방법


## PyPDFLoader


가장 일반적으로 많이 활용되는 방법이며, 대부분의 일반적인 PDF 파일을 문제 없이 불러 올 수 있습니다.


### ① loader.load()

문서를 페이지 단위로 불러옵니다. `document` 변수에는 페이지 별 `Document` 객체가 리스트 형태로 존재하며, **1개 Document == PDF 문서의 1개 페이지** 를 의미합니다.


In [2]:
# 파일경로
filepath = "data/상품요약서_신한스포츠&레저보장보험Plus(무배당)_240401_P9.pdf"

In [3]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader(filepath)

# 페이지 별 문서 로드
document = loader.load()

print(f"문서의 수: {len(document)}")

문서의 수: 9


로드한 `document` 의 내용을 확인해 보면 다음과 같습니다.


In [4]:
# page_content 에는 본문의 내용이 있음
print(document[0].page_content[:200])  # 일부내용 출력
print("===" * 10)
# metadata 출력
print(document[0].metadata)

신한스포츠 &레저보장보험 Plus( 무배당 )
상  품  요  약  서
이 상품요약서 는 보험약관 등 신한스포츠 &레저보장보험 Plus( 무배당 )의 기초
서류에 기재된 주요내용을 요약한 것이므로 구체적인 내용은  반드시 보험약
관 등을 참조하시기 바랍니다 .
{'source': 'data/상품요약서_신한스포츠&레저보장보험Plus(무배당)_240401_P9.pdf', 'page': 0}


In [5]:
# PDF문서의 페이지 수 == len(document)
print(f"문서의 수: {len(document)}")

문서의 수: 9


### ② TextSplitter 활용


문서를 로드시 페이지 단위로 분리 저장하는 방법이 아닌, chunk_size 단위로 저장할 수 있는데, chunk_size 단위로 분리/저장하기 위해서는 다음과 같이 `TextSplitter` 를 활용할 수 있습니다.

- `chunk_size`: 하나의 chunk(단위) 당 보관할 토큰 수
- `chunk_overlap`: chunk 간 겹쳐지는 토큰의 개수


In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader(filepath)
print(loader)

### CharacterTextSplitter

- [API 문서](https://api.python.langchain.com/en/stable/text_splitter/langchain.text_splitter.CharacterTextSplitter.html?highlight=charactertext#langchain.text_splitter.CharacterTextSplitter)

가장 간단한 방법입니다. 이 방법은 문자(기본값은 "")를 기준으로 분할하고 문자 수에 따라 Chunk 의 길이를 측정합니다.

- 텍스트 분할 방법: Character
- Chunk 크기 측정 방법: Character


In [7]:
text = "안녕하세요. 반갑습니다. 제 이름은 테디입니다.\n\n\n 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다."

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator=".")
text_splitter.split_text(text)

['안녕하세요. 반갑습니다. 제 이름은 테디입니다.\n\n\n 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다']

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=10, separator=" ")
text_splitter.split_text(text)

['안녕하세요. 반갑습니다. 제 이름은 테디입니다.\n\n\n 랭체인은 정말 좋은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은',
 '랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은 테디입니다.',
 '이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제 이름은',
 '제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요. 반갑습니다. 제',
 '반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다. 안녕하세요.',
 '안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.']

In [10]:
# splitter 정의
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# 문서 로드 및 분할 (load_and_split)
split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"CharacterTextSplitter \t\t 사용시 문서의 수: {len(split_docs)}")

CharacterTextSplitter 		 사용시 문서의 수: 9


In [11]:
split_docs[3].page_content

'4 신한스포츠 &레저보장보험 Plus(무배당 )Ⅱ 보험금 지급사유 및 지급제한 사항\n1. 상품의 구성\n주  계  약 신한스포츠 &레저보장보험 Plus( 무배당 )\n제도성 특약 + 장애인전용보험전환특약\n2. 보험금 지급내용\n※ 보장관련 유의사항\n- 해당 상품에서 정한 보장 여부의 판단은 질병의 진단 및 재해 발생당시의 한국표준질병 ·\n사인분류를 기준으로 하며, 이후 한국표준질병 ·사인분류가 개정되는 경우에는 질병의 진\n단 및 재해 발생 당시에 시행되고 있는 한국표준질병 ·사인분류를 기준으로 판단합니다 .\n- 감염병에 관한 법률이 제·개정될 경우, 보험사고 발생당시 제·개정된 법률을 적용합니다 . \n(자세한 내용은 약관 "재해분류표 " 또는 "질병 및 재해 분류표 " 참조)\n- “재해골절치료급여금 ”에서 재해골절 (치아파절 제외)은 치아의 파절(분류번호 S02.5) 을 \n제외합니다 .\n- 보험금의 지급사유가 중복하여 발생한 경우에는 각각에 해당하는 보험금을 지급합니다 .\n \n(1)주계약\n구 분 지  급  사  유 지  급  금  액\n중대한재해\n수술급여금보험기간 중 피보험자가 재해로 인한 그 직접적인 \n치료를 목적으로 중대한 재해수술을 받았을 때(재\n해수술급여금에 추가지급 )【수술 1회당】\n보험가입금액의 30%\n아킬레스\n힘줄손상\n수술급여금보험기간 중 「피보험자에게 재해가 발생하고 그 \n재해를 직접적인 원인으로 아킬레스힘줄손상으로 \n진단확정되고 , 그 직접적인 치료를 목적으로 수술」\n을 받았을 때(재해수술급여금에 추가지급 )【수술 1회당】\n보험가입금액의 4%\n재해 보험기간 중 피보험자가 재해로 인한 그 직접적인 【수술 1회당】'

In [12]:
# splitter 정의
text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50, separator="\n")

# 문서 로드 및 분할 (load_and_split)
split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"CharacterTextSplitter \t\t 사용시 문서의 수: {len(split_docs)}")

CharacterTextSplitter 		 사용시 문서의 수: 12


In [13]:
split_docs[5].page_content

'재해골절\n치료급여금보험기간 중 피보험자에게 재해가 발생하고 그 재\n해를 직접적인 원인으로 재해골절 (치아파절 제외)\n로 진단이 확정되었을 때【골절 1회당】\n보험가입금액의 1%\n5대재해골절\n치료급여금보험기간 중 피보험자에게 재해가 발생하고 그 재\n해를 직접적인 원인으로 5대재해골절 (비골의 골절 \n및 치아파절 제외)로 진단이 확정되었을 때\n※ 5대재해골절치료급여금 지급사유 발생시 재해\n골절치료급여금 추가로 지급【골절 1회당】\n보험가입금액의 4%\n※ 중대한 재해수술 : 재해에 의한 개두수술 , 개흉수술 , 개복수술'

### RecursiveCharacterTextSplitter

- [참고문서](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter)
- [API 문서](https://api.python.langchain.com/en/stable/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html?highlight=charactertext)

이 텍스트 분할기는 일반 텍스트에 권장되는 텍스트 분할기입니다. 문자 목록으로 매개변수화됩니다. 청크가 충분히 작아질 때까지 순서대로 분할을 시도합니다. 기본 목록은 `["\n\n", "\n", " ", ""]` 입니다.

이렇게 하면 일반적으로 의미적으로 가장 연관성이 강한 텍스트 조각으로 보이는 모든 단락(그리고 문장, 단어)을 가능한 한 길게 유지하려는 효과가 있습니다.

- 텍스트를 분할하는 방법: **list of characters**
- Chunk 크기 측정 방법: **number of characters**


In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len,
)

# 문서 로드 및 분할 (load_and_split)
split_doc = loader.load_and_split(text_splitter=text_splitter)
print(f"RecursiveCharacterTextSplitter \t 사용시 문서의 수: {len(split_doc)}")

RecursiveCharacterTextSplitter 	 사용시 문서의 수: 12


In [15]:
# text = "안녕하세요. 반갑습니다. 제 이름은 테디입니다.\n\n\n 랭체인은 정말 좋은 프로젝트입니다.안녕하세요. 반갑습니다. 제 이름은 테디입니다. 랭체인은 정말 좋은 프로젝트입니다.안녕하세요."
text = """
오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented Generation (RAG)'에 대한 것입니다.

Retrieval Augmented Generation 

기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다. 
이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다. 
이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게 됩니다. 

RAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다. 
이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다.
"""

text_splitter1 = CharacterTextSplitter(
    chunk_size=50, chunk_overlap=10, separator=" ")
text_splitter1.split_text(text)

["오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented",
 "Augmented Generation (RAG)'에 대한 것입니다.\n\nRetrieval",
 'Augmented Generation \n\n기존의 언어 모델에 검색 기능을 통합하여 성능을',
 '통합하여 성능을 향상시키는 방법론입니다. \n이 접근 방식에서, 모델은 먼저 관련 정보를',
 '먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다. \n이를 통해 모델은 더',
 '통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게',
 '제공할 수 있게 됩니다. \n\nRAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한',
 '특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다. \n이 기술은 자연어 처리 분야에서',
 '처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등',
 '답변 시스템 등 다양한 응용 분야에 활용되고 있습니다.']

In [16]:
text_splitter2 = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len,
)
text_splitter2.split_text(text)

["오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented",
 "Augmented Generation (RAG)'에 대한 것입니다.",
 'Retrieval Augmented Generation',
 '기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다.',
 '이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을',
 '바탕으로 응답을 생성합니다.',
 '이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수',
 '답변을 제공할 수 있게 됩니다.',
 'RAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데',
 '제공하는 데 유용합니다.',
 '이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약,',
 '문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다.']

### TokenTextSplitter


In [17]:
from langchain.text_splitter import TokenTextSplitter

# splitter 정의
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = loader.load_and_split(text_splitter=text_splitter)
print(f"TokenTextSplitter \t\t 사용시 문서의 수: {len(split_docs)}")

TokenTextSplitter 		 사용시 문서의 수: 17


In [18]:
text_splitter3 = TokenTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
)
text_splitter3.split_text(text)

["\n오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented Generation (RAG",
 "� 'Retrieval Augmented Generation (RAG)'에 대한 것입니다.\n\nRetrieval Augmented Generation \n\n기존의",
 '\n\n기존의 언어 모델에 검색 기능을 통합하�',
 '� 통합하여 성능을 향상시키는 방법론입니',
 '��론입니다. \n이 접근 방식에서, 모델은 먼저',
 '��은 먼저 관련 정보를 검색하고, 그 정보�',
 '그 정보를 바탕으로 응답을 생성합니다. \n이�',
 '�다. \n이를 통해 모델은 더 정확하고, 상세하�',
 ' 상세하며, 실제 세계의 지식에 기반한 답변',
 '��한 답변을 제공할 수 있게 됩니다. \n\nRAG는 특�',
 'RAG는 특히 복잡한 질문에 답하거나, 특정 주',
 ' 특정 주제에 대한 상세한 설명을 제공하는 �',
 '공하는 데 유용합니다. \n이 기술은 자연어 처�',
 '연어 처리 분야에서 중요한 진전을 이루고 �',
 ' 이루고 있으며, 정보 검색, 문서 요약, 질�',
 '��약, 질문 답변 시스템 등 다양한 응용 분야',
 '�용 분야에 활용되고 있습니다.\n']

In [19]:
from langchain.text_splitter import KonlpyTextSplitter

text_splitter4 = KonlpyTextSplitter()
text_splitter4.split_text(text)

["오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented Generation (RAG) '에 대한 것입니다.\n\nRetrieval Augmented Generation 기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다.\n\n이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다.\n\n이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게 됩니다.\n\nRAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다.\n\n이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다."]

In [20]:
print(split_docs[10].page_content)

��한 날의 다음 날부터 반환일까지의 기간에 
대하여 회사는 이 계약의 보험계약대출이율을 연단위 복리로 계산한 금액을 더하여 돌려 드립니다 .
2. “1.”에 따라 계약을 취소한 경우 회사는 보험금을 지급할 책임이 없고, 이미 지급한 보험금의 반환을 청
구할 수 있습니다 .
3. “1.”에 따라 계약을 취소한 경우 회사는 계약자에게 실제 납입한 보험료 누계액 (다만, “감액 등으로 회사
가 실제 지급한 금액”을 차감한 금액)을 돌려드립니다 . 다만, 보험료를 받은 기간에 대한 이자는 지급하지 
않습니다 .
    ④ 계약 전 알릴 의무 관련사항
   1. 가입자의 계약 전 알릴 의무
   계약자 또는 피보험자는 청약시 (진단계약의 경우에는 건강진단할 때) 청약서에서 질문한 사항에 대하여 
알고 있는 사실을 반드시 사실대로 알려야 (이하 “계약 전 알릴 의무”라 하며, 상법상 “고지의무 ”와 같
습니다 )합니다 . 다만, 진단계약에서 의료법 제3조(의료기관 )의 규정에 따른 종합병원 및 �


## Semantic Chunker


In [21]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

In [22]:
text_splitter4 = SemanticChunker(
    OpenAIEmbeddings()
)

In [23]:
text_splitter4.split_text(text)

["\n오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented Generation (RAG)'에 대한 것입니다. Retrieval Augmented Generation \n\n기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다. 이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다.",
 '이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게 됩니다. RAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다. 이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다. ']

In [24]:
for sent in text_splitter4.split_text(text):
    print(sent)


오늘 여러분께 소개해 드릴 내용은 'Retrieval Augmented Generation (RAG)'에 대한 것입니다. Retrieval Augmented Generation 

기존의 언어 모델에 검색 기능을 통합하여 성능을 향상시키는 방법론입니다. 이 접근 방식에서, 모델은 먼저 관련 정보를 검색하고, 그 정보를 바탕으로 응답을 생성합니다.
이를 통해 모델은 더 정확하고, 상세하며, 실제 세계의 지식에 기반한 답변을 제공할 수 있게 됩니다. RAG는 특히 복잡한 질문에 답하거나, 특정 주제에 대한 상세한 설명을 제공하는 데 유용합니다. 이 기술은 자연어 처리 분야에서 중요한 진전을 이루고 있으며, 정보 검색, 문서 요약, 질문 답변 시스템 등 다양한 응용 분야에 활용되고 있습니다. 
